In [1]:
import pyanitools as pyt
import pyaniasetools as aat
import numpy as np
import hdnntools as hdt
import os

import matplotlib.pyplot as plt

In [2]:
file_old = '/home/jujuman/Research/DataReductionMethods/h5_rmlow_error/h5_old/'
file_new = '/home/jujuman/Research/DataReductionMethods/h5_rmlow_error/h5_new/'

wkdir = '/home/jujuman/Research/DataReductionMethods/h5_stats_check/ANI-AL-0707.0001.0410'
#wkdir = '/home/jujuman/Research/ANI-validation/'
cnstfile = wkdir + '/train0/rHCNOSFCl-4.6A_16-3.1A_a4-8.params'
saefile  = wkdir + '/train0/sae_wb97x-631gd.dat'
nnfdir   = wkdir + '/train'
Nn = 5

In [3]:
files = os.listdir(file_old)
files.sort()

# Define the conformer cross validator class
anicv = aat.anicrossvalidationconformer(cnstfile,saefile,nnfdir,Nn,1,False)

In [4]:
Nt = 0
Nk = 0
for k,f in enumerate(files):
    print('Working on file:',f)
    adl = pyt.anidataloader(file_old+f)

    # Data storage
    dpack = pyt.datapacker(file_new+f, mode='w')

    for i,data in enumerate(adl):
        #if i == 20:
        #    break
        X = data['coordinates']
        S = data['species']
        Edft = hdt.hatokcal * data['energies']
        path = data['path']
        del data['path']
    
        Eani, Fani = anicv.compute_energy_conformations(X=np.array(X,dtype=np.float32),S=S)
    
        Eani_m = np.mean(Eani, axis=0)
        #Fani = np.mean(Fani, axis=0)
    
        err = Eani_m - Edft 
        pae = np.abs(err)/np.sqrt(float(len(S)))
        idx = np.where(pae > 0.15)
        
        Nt += err.size
        Nk += idx[0].size
        
        klist = ['cm5', 'hirshfeld', 'hirdipole', 'forces', 'coordinates', 'spindensities', 'energies']
        #klist = ['CM5', 'hirshfeld', 'forces', 'coordinates', 'energies']
        #print(data.keys())
        
        data_new = data.copy()
        for key in klist:
            data_new[key] = data[key][idx]
            #print(key,type(data[key][0]),type(data_new[key][0]),type(data[key][gidx][0]))
            
        dpack.store_data(path, **data_new)
    print('done')
    dpack.cleanup()
print(Nk,'/',Nt,' = ',100.0*Nk/float(Nt))

Working on file: confs_cv_gdb01-08_rs1.h5
done
Working on file: confs_cv_gdb01-08_rs2.h5
done
Working on file: confs_cv_gdb01-08_rs3.h5
done
Working on file: confs_cv_gdb01-08_rs4.h5
done
708836 / 810908  =  87.41262880622709


In [5]:
bi = np.where(pae > 0.15)

Nt = pae.size
Nb = bi[0].size


print(Nb,'/',Nt,' = ',Nb/float(Nt))

6 / 8  =  0.75


In [6]:
adl = pyt.anidataloader(file_to)
for data in adl:
    X = data['coordinates']
    S = data['species']
    Edft = hdt.hatokcal * data['energies']
    
    Eani, Fani = anicv.compute_energy_conformations(X=X,S=S)
    
    Eani = np.mean(Eani, axis=0)
    Fani = np.mean(Fani, axis=0)
    
    print(hdt.calculaterootmeansqrerror(Eani,Edft))
    

NameError: name 'file_to' is not defined